# Data Transformation

## will use Titanic dataset to clean the data and transform the data using different methods

In [1]:
!pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn import datasets
data = datasets.fetch_openml(name='Titanic', version=1, as_frame=True)
titanic = data.frame
titanic

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [4]:
pd.isnull(titanic).sum()
titanic.drop(labels=['cabin','boat','body','home.dest'],axis=1, inplace=True)

In [5]:
titanic.sample(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked
165,1,1,"Hoyt, Mr. Frederick Maxfield",male,38.0,1,0,19943,90.0000,S
827,3,0,"Goodwin, Master. William Frederick",male,11.0,5,2,CA 2144,46.9000,S
1048,3,1,"Nakid, Miss. Maria ('Mary')",female,1.0,0,2,2653,15.7417,C
707,3,0,"Carlsson, Mr. August Sigfrid",male,28.0,0,0,350042,7.7958,S
692,3,0,"Buckley, Miss. Katherine",female,18.5,0,0,329944,7.2833,Q
127,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
760,3,1,"de Mulder, Mr. Theodore",male,30.0,0,0,345774,9.5000,S
104,1,1,"Eustis, Miss. Elizabeth Mussey",female,54.0,1,0,36947,78.2667,C
859,3,0,"Hegarty, Miss. Hanora 'Nora'",female,18.0,0,0,365226,6.7500,Q
449,2,1,"Hocking, Mrs. Elizabeth (Eliza Needs)",female,54.0,1,3,29105,23.0000,S


In [6]:
print(titanic.dtypes)
print("-------------------")
print(titanic['ticket'].value_counts())
titanic.drop(labels=['ticket'], axis=1, inplace=True)

pclass         int64
survived    category
name          object
sex         category
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
embarked    category
dtype: object
-------------------
ticket
CA. 2343        11
CA 2144          8
1601             8
347082           7
S.O.C. 14879     7
                ..
349202           1
347466           1
347075           1
350043           1
348125           1
Name: count, Length: 929, dtype: int64


In [7]:
titanic
titanic.isnull().sum()
titanic['fare'].fillna(titanic['fare'].mean(), inplace=True)
titanic['embarked'].fillna('S', inplace=True)
titanic.isnull().sum()
titanic['embarked'].value_counts()

/tmp/ipykernel_6036/462100465.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['fare'].fillna(titanic['fare'].mean(), inplace=True)
/tmp/ipykernel_6036/462100465.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

embarked
S    916
C    270
Q    123
Name: count, dtype: int64

In [8]:
titanic.isnull().sum()

pclass        0
survived      0
name          0
sex           0
age         263
sibsp         0
parch         0
fare          0
embarked      0
dtype: int64

### From here will use sklearn to clean the data

In [9]:
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split

In [10]:
X = titanic.drop("survived", axis=1)
Y = titanic['survived']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=0.2)
X_train

,pclass,name,sex,age,sibsp,parch,fare,embarked
772,3,"Dika, Mr. Mirko",male,17.0,0,0,7.8958,S
543,2,"Reeves, Mr. David",male,36.0,0,0,10.5000,S
289,1,"Taussig, Miss. Ruth",female,18.0,0,2,79.6500,S
10,1,"Astor, Col. John Jacob",male,47.0,1,0,227.5250,C
147,1,"Harrington, Mr. Charles H",male,NaN,0,0,42.4000,S
...,...,...,...,...,...,...,...,...
1095,3,"O'Sullivan, Miss. Bridget Mary",female,NaN,0,0,7.6292,Q
1130,3,"Pettersson, Miss. Ellen Natalia",female,18.0,0,0,7.7750,S
1294,3,"Williams, Mr. Leslie",male,28.5,0,0,16.1000,S
860,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S


In [11]:
imp = SimpleImputer(missing_values=np.nan, strategy="mean")
X_train['age'] = imp.fit_transform(X_train[['age']])
X_test['age'] = imp.transform(X_test[['age']])

In [12]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import StandardScaler

In [13]:
# column sex, embarked --> nominal category
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_Sex_embarked = ohe.fit_transform(X_train[['sex', 'embarked']])
X_test_sex_embarked = ohe.transform(X_test[['sex', 'embarked']])

In [14]:
X_train.drop('name', inplace=True, axis=1)
X_test.drop('name', inplace=True, axis=1)

In [15]:
X_train_Sex_embarked

array([[1., 0., 1.],
       [1., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], shape=(1047, 3))

In [16]:
train_stacked = np.hstack((X_train[['pclass','age','sibsp','parch','fare']].values, X_train_Sex_embarked))
test_stacked = np.hstack((X_test[['pclass','age','sibsp','parch','fare']].values, X_test_sex_embarked))
test_stacked

array([[ 3.       , 35.       ,  0.       , ...,  1.       ,  0.       ,
         1.       ],
       [ 3.       , 20.       ,  1.       , ...,  1.       ,  0.       ,
         0.       ],
       [ 3.       , 29.5327381,  0.       , ...,  1.       ,  0.       ,
         1.       ],
       ...,
       [ 3.       , 21.       ,  0.       , ...,  0.       ,  0.       ,
         1.       ],
       [ 1.       , 50.       ,  1.       , ...,  1.       ,  0.       ,
         0.       ],
       [ 1.       , 24.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], shape=(262, 8))

In [17]:
ss = StandardScaler()
X_train_transformed = ss.fit_transform(train_stacked)
X_test_transformed = ss.transform(test_stacked)
X_test_transformed

array([[ 8.40358979e-01,  4.28120063e-01, -4.95964298e-01, ...,
         7.39275864e-01, -3.24956337e-01,  6.47069693e-01],
       [ 8.40358979e-01, -7.46471726e-01,  4.56833170e-01, ...,
         7.39275864e-01, -3.24956337e-01, -1.54542859e+00],
       [ 8.40358979e-01,  7.78957819e-15, -4.95964298e-01, ...,
         7.39275864e-01, -3.24956337e-01,  6.47069693e-01],
       ...,
       [ 8.40358979e-01, -6.68165607e-01, -4.95964298e-01, ...,
        -1.35267503e+00, -3.24956337e-01,  6.47069693e-01],
       [-1.55055366e+00,  1.60271185e+00,  4.56833170e-01, ...,
         7.39275864e-01, -3.24956337e-01, -1.54542859e+00],
       [-1.55055366e+00, -4.33247249e-01, -4.95964298e-01, ...,
        -1.35267503e+00, -3.24956337e-01, -1.54542859e+00]],
      shape=(262, 8))

In [18]:
X_test_final = pd.DataFrame(X_test_transformed)
X_test_final.columns = ['pclass', 'age', 'sibp', 'parch','fare', 'male', 'c', 's']
X_train_final = pd.DataFrame(X_train_transformed)
X_train_final.columns = ['pclass', 'age', 'sibp', 'parch','fare', 'male', 'c', 's']
X_train_final

,pclass,age,sibp,parch,fare,male,c,s
0,0.840359,-9.813901e-01,-0.495964,-0.442432,-0.495577,0.739276,-0.324956,0.647070
1,-0.355097,5.064262e-01,-0.495964,-0.442432,-0.445264,0.739276,-0.324956,0.647070
2,-1.550554,-9.030840e-01,-0.495964,1.795376,0.890704,-1.352675,-0.324956,0.647070
3,-1.550554,1.367793e+00,0.456833,-0.442432,3.747629,0.739276,-0.324956,-1.545429
4,-1.550554,7.789578e-15,-0.495964,-0.442432,0.171040,0.739276,-0.324956,0.647070
...,...,...,...,...,...,...,...,...
1042,0.840359,7.789578e-15,-0.495964,-0.442432,-0.500728,-1.352675,3.077337,-1.545429
1043,0.840359,-9.030840e-01,-0.495964,-0.442432,-0.497911,-1.352675,-0.324956,0.647070
1044,0.840359,-8.086971e-02,-0.495964,-0.442432,-0.337073,0.739276,-0.324956,0.647070
1045,0.840359,-2.766350e-01,-0.495964,-0.442432,-0.495013,-1.352675,-0.324956,0.647070


In [19]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
model = DecisionTreeClassifier()
model.fit(X_train_final, Y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [25]:
model.score(X_test_final, Y_test)

0.767175572519084

## will use column transformer with pipleines to do the same thing

In [30]:
import pandas as pd
import numpy as np
from sklearn import datasets
data = datasets.fetch_openml(name='Titanic', version=1, as_frame=True)
titanic = data.frame
titanic

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [31]:
titanic.drop(labels=["name", "cabin","ticket", "boat","body", "home.dest"], axis=1, inplace=True)
titanic.value_counts()

pclass  survived  sex   age   sibsp  parch  fare     embarked
2       0         male  30.0  0      0      13.0000  S           5
3       0         male  17.0  0      0      8.6625   S           4
2       0         male  23.0  0      0      13.0000  S           3
3       0         male  21.0  0      0      8.0500   S           3
2       0         male  25.0  0      0      13.0000  S           3
                                                                ..
                        29.0  1      0      26.0000  S           1
                                            27.7208  C           1
                        30.0  0      0      10.5000  S           1
                              1      0      21.0000  S           1
                        25.0  1      2      41.5792  C           1
Name: count, Length: 971, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(titanic.drop(labels="survived", axis=1), titanic["survived"],random_state=42, test_size=0.2)
X_train

,pclass,sex,age,sibsp,parch,fare,embarked
772,3,male,17.0,0,0,7.8958,S
543,2,male,36.0,0,0,10.5000,S
289,1,female,18.0,0,2,79.6500,S
10,1,male,47.0,1,0,227.5250,C
147,1,male,NaN,0,0,42.4000,S
...,...,...,...,...,...,...,...
1095,3,female,NaN,0,0,7.6292,Q
1130,3,female,18.0,0,0,7.7750,S
1294,3,male,28.5,0,0,16.1000,S
860,3,female,26.0,0,0,7.9250,S


In [39]:
trf1 = ColumnTransformer([
    ("impute_age", SimpleImputer(), [2]),
    ("impute_fare", SimpleImputer(), [5]),
    ("impute_embarked", SimpleImputer(strategy="most_frequent"), [6])
], remainder='passthrough')

In [40]:
trf2 = ColumnTransformer([
    ("encode_sex", OneHotEncoder(drop='first', sparse_output=False), [1]),
    ("encode_embarked", OneHotEncoder(drop='first', sparse_output=False), [6])
], remainder='passthrough')

In [ ]:
trf3 = ColumnTransformer([
    ()
])